In [1]:
using OrdinaryDiffEq
using SciMLSensitivity
using Enzyme
using ComponentArrays # 方便管理参数
using LinearAlgebra
using Parameters

In [2]:
abstract type AbstractSoilParam{T} end

# Van Genuchten 参数
@with_kw struct VGParam{T} <: AbstractSoilParam{T}
    θ_s::T
    θ_r::T
    Ks::T  # [cm/h]
    α::T   # [1/cm]
    n::T   # [-]
end

# 土壤配置（常量）
struct SoilConfig{T, I}
    ibeg::I
    N::I
    Δz::Vector{T}    # [cm]
    Δz_half::Vector{T} # [cm]
    depths::Vector{T}
end

# 土壤缓存（预分配内存，用于 In-place 计算）
struct SoilCache{T}
    K::Vector{T}
    K_half::Vector{T}
    ψ::Vector{T}
    Q::Vector{T}
    sink::Vector{T}
end

# 构造函数
function make_soil(n_layers::Int, dz_val::T) where T
    depths = collect(range(dz_val, length=n_layers, step=dz_val))
    Δz = fill(dz_val, n_layers)
    # 简单的交界面距离计算
    Δz_half = copy(Δz) 
    
    cfg = SoilConfig(1, n_layers, Δz, Δz_half, depths)
    
    cache = SoilCache(
        zeros(T, n_layers),      # K
        zeros(T, n_layers),      # K_half (interface)
        zeros(T, n_layers),      # ψ
        zeros(T, n_layers + 1),  # Q (N+1 interfaces)
        zeros(T, n_layers)       # sink
    )
    return cfg, cache
end

# --- 2. 物理方程实现 ---

# Van Genuchten - Mualem 模型
# 使用 inline 帮助编译器优化
@inline function vg_hydraulic(θ, p::VGParam{T}) where T
    S_e = (θ - p.θ_r) / (p.θ_s - p.θ_r)
    S_e = clamp(S_e, T(1e-3), T(1.0 - 1e-3)) # 避免 log(0) 或除 0
    
    m = 1 - 1/p.n
    # 计算 ψ (吸力 head, cm)
    # S_e = [1 + (α|ψ|)^n]^(-m) -> |ψ| = 1/α * (S_e^(-1/m) - 1)^(1/n)
    head = - (1/p.α) * (S_e^(-1/m) - 1)^(1/p.n)
    
    # 计算 K (导水率, cm/h)
    # K = Ks * S_e^0.5 * [1 - (1 - S_e^(1/m))^m]^2
    denom = (1 - S_e^(1/m))^m
    k_val = p.Ks * sqrt(S_e) * (1 - denom)^2
    
    return k_val, head
end

function update_hydraulic_props!(cache, θ, ps)
    @inbounds for i in eachindex(θ)
        K, ψ = vg_hydraulic(θ[i], ps[i])
        cache.K[i] = K
        cache.ψ[i] = ψ
    end
    # 计算交界面 K (算术平均或几何平均)
    N = length(cache.K)
    @inbounds for i in 1:N-1
        # 简单的算术平均
        cache.K_half[i] = (cache.K[i] + cache.K[i+1]) / 2 
    end
    return nothing
end

# 简单的上边界条件
function boundary_top(t, method)
    # 假设恒定降雨 或 基于时间的函数
    return -1.0 # [cm/h] 向下为负
end

# --- 3. 核心逻辑：Richards Equation ---

function cal_Q!(cache::SoilCache{T}, config::SoilConfig, ps, θ, t) where T
    N = config.N
    
    # 1. 更新 K 和 ψ
    update_hydraulic_props!(cache, θ, ps)

    # 2. 上边界处理
    Q_top = boundary_top(t, "flux")
    cache.Q[1] = Q_top

    # 3. 内部通量 (Darcy Law)
    # Q = -K * (dψ/dz + 1)
    @inbounds for i in 1:N-1
        dψ_dz = (cache.ψ[i+1] - cache.ψ[i]) / config.Δz_half[i] # 注意这里方向需根据坐标系确认
        # 假设 z 向下为正，depth 增加。 
        # 若 ψ 随深度增加变小（更负），则 dψ/dz < 0。
        # 水往势能低处流。公式通常为 Q = -K (d(ψ-z)/dz) = -K(dψ/dz - 1) (若z向上为正)
        # 这里简化假设：
        q_val = -cache.K_half[i] * ( (cache.ψ[i] - cache.ψ[i+1])/config.Δz_half[i] + 1.0 )
        cache.Q[i+1] = q_val
    end
    # 4. 下边界 (自由排水: 梯度为0, 只有重力)
    cache.Q[N+1] = -cache.K[N] 
    return nothing
end

# 适配 SciML 的 ODE 函数签名 f(du, u, p, t)
# p 包含: (parameters, config, cache)
function richards_eq!(dθ, θ, ps, t, config, cache)
    # ps 是我们需要求导的土壤参数
    # config 是常量
    # cache 是预分配内存 (Enzyme 会自动处理它的 Duplicated/Const 状态)    
    
    # 1. 计算通量 Q (In-place 修改 cache)
    cal_Q!(cache, config, ps, θ, t)
    
    # 2. 计算 dθ/dt = -dQ/dz
    # 单位换算: 假设 t 是 [h], Flux 是 [cm/h], Δz 是 [cm], dθ 是 [-]
    # 则 dθ/dt = [1/h]. 如果 ODE 求解器 t 是秒, 则需要 /3600.
    # 这里假设 t 单位就是 小时。
    ibeg = config.ibeg
    N = config.N
    
    # 第一层
    dθ[1] = -(cache.Q[2] - cache.Q[1]) / config.Δz[1]
    # 中间层
    @inbounds for i in 2:N
        dθ[i] = -(cache.Q[i+1] - cache.Q[i]) / config.Δz[i]
    end
    return nothing
end

richards_eq! (generic function with 1 method)

In [3]:
n_layers = 10
dz = 5.0 # cm
config, cache = make_soil(n_layers, dz)
Equation!(dθ, θ, ps, t) = richards_eq!(dθ, θ, ps, t, config, cache)

θ0 = fill(0.3, n_layers)

# 真实参数 (Target)
p_true_struct = [VGParam(0.45, 0.05, 10.0, 0.01, 2.0) for _ in 1:n_layers]
# 猜测参数 (Start) -> 我们要对这个求导
p_init_struct = [VGParam(0.40, 0.04, 8.0, 0.012, 1.8) for _ in 1:n_layers]

# 封装参数到 ComponentArray 以便向量化处理（SciML 友好）
# 注意：为了让 Enzyme 正常工作，最外层最好是 Array 或 ComponentArray
# 这里为了演示简单，我们把 p 包装成 NamedTuple 传入 ODE
tspan = (0.0, 10.0) # 10小时
# 2. 定义 Loss 函数
# 我们希望优化 p_guess 使最终含水量接近 target
function loss_function(p_params_arr)
    # 重构参数结构体 (这里简化为每一层参数相同，方便演示)
    # 实际应用中可以用 ComponentArrays
    val_Ks = p_params_arr[1]
    val_n  = p_params_arr[2]
    ps_current = [VGParam(0.45, 0.05, val_Ks, 0.01, val_n) for _ in 1:n_layers]
    # 构建 p 组合
    p_ode = (; ps=ps_current, config=config, cache=cache)
    
    # 创建 Problem
    prob = ODEProblem(richards_eq!, θ0, tspan, p_ode)
    
    # --- 关键部分：指定 Enzyme 的 Sensitivity Algorithm ---
    # InterpolatingAdjoint: 检查点 + 插值
    # autojacvec=EnzymeVJP(): 内部 VJP 计算使用 Enzyme
    sensalg = InterpolatingAdjoint(autojacvec=EnzymeVJP())
    sol = solve(prob, Tsit5(), saveat=1.0, sensealg=sensalg)
    # 简单的 Loss: 最小化最后时刻所有层含水量的和 (只是为了产生梯度)
    return sum(sol[end])
end

# 3. 执行微分
# 输入: [Ks, n]
p_in = [8.0, 1.8] 

# 分配梯度内存
dp = zeros(length(p_in))
println("Starting Enzyme Autodiff...")

# Active: 返回值是 Loss
# Duplicated: p_in 是需要求导的数组
Enzyme.autodiff(Enzyme.Reverse, loss_function, Enzyme.Active, Enzyme.Duplicated(p_in, dp))

println("Gradient calculation finished.")
println("dLoss/dKs = ", dp[1])
println("dLoss/dn  = ", dp[2])

Starting Enzyme Autodiff...


LoadError: "Error cannot store inactive but differentiable variable SoilConfig{Float64, Int64}(1, 10, [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0], [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0], [5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0]) into active tuple"